<!--<a href="https://colab.research.google.com/github/mkierczak/autoencoders_workshop/blob/main/PCA2VAE_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>-->

# Initial Setup

First, we need to import appropriate libraries that we will use throughout the workshop.

Note! Some of the libraries below are not directly used in the code below but will be necessary once you start experimenting.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
from keras import layers
from keras import backend as K
from keras.regularizers import l1
from matplotlib import pyplot as plt
from keras.utils import plot_model
from keras import backend
from sklearn.decomposition import PCA
from itertools import product

# Data

We will be working on a dataset that I have created from HapMap phase 3 project. In HapMap, several individuals have been genotyped that come from different parts of the world and from different ethnic backgrounds. For the sake of simplicity, I have **randomly selected 5000 autosomal** markers that we will be working with and saved them along with phenotype data (biological sex, ethnicity etc.) and a pre-computed genomic kinship matrix in the *hdf5* file format.

If you want to try another dataset, you can go for e.g., data from Lazaridis et. al. [*Genomic insights into the origin of farming in the ancient Near East*](https://www.nature.com/articles/nature19310) 2016. Nature **536**:419-424. I have pre-prepared chr1 markers at call rate of 0.99 and higher.

In [ ]:
# HapMap3 randomly selected 5000 autosomal markers data
!wget -O data.hd5 https://www.dropbox.com/scl/fi/2daedhwkdjweotnthxee3/HapMap3_5000.h5?rlkey=nz8f9df7tt9n0hrpqg7y19omc&dl=1

# Data for chr1 from Lazaridis et al.
#!wget -O data.hd5 https://www.dropbox.com/scl/fi/kck4puyi1qmuzr65bgdbn/HumanOriginsPublic2068_geno_chr1.h5?rlkey=xp4nfljz0c2za9ihriletxx3x&dl=1


In [ ]:
# Extract genotypes, phenotypes and genomic kinship matrix from hdf5 file
orig_geno = pd.read_hdf('data.hd5', key = 'geno')
orig_pheno = pd.read_hdf('data.hd5', key = 'pheno')
orig_gkin = pd.read_hdf('data.hd5', key = 'gkin')


In [ ]:
# Let's have a look at the genotypes data
orig_geno.info()
orig_geno.iloc[0:4, 0:4]

In [ ]:
# Examine phenotypes
orig_pheno.iloc[0:4, ]

In [ ]:
# Look at the genomic kinship matrix
orig_gkin.iloc[0:4, ]

In [ ]:
# Check whether we have missing genotypes in our data
print("Missing genotypes per marker: \n", orig_geno.isna().sum())

Since neural networks require numeric values as input, we do not need to do anything -- our genotypes are already encoded as minor allele counts. However, we know that it is best for the networks to get input from the range [0, 1], thus we shall re-code our genotypes so that they are bound between zero and one. 

Now, one of the alleles is encoded as 0, meaning that for this particular genotype the bias term can take over the weight (y = 0*w_0 + b). While true in previous implementations of tensorflow, this "bias hitchhiking" is no longer a concern.

In [ ]:
geno = orig_geno.replace([0, 1.0, 2.0], [0, 0.5, 1.0])
#geno.fillna(0, inplace = True)
#print(geno.isna().sum())
geno


# Training Phase

Now, we will follow some best machine learning practises and randomly split our data into two sets: the training set and the test set. The training set will be used to train our neural network, i.e. to optimize network weights while the latter will be set aside so that it is NEVER used in the training phase. This set will be used to objectively assess performance of our model at the very end.

## Question

Do you think that this random split of the original dataset is good enough?



In [ ]:
train = geno.sample(frac = 0.8, random_state = 42) # 80% of our individuals will go to the training set
test = geno.drop(train.index)
pheno = orig_pheno.set_index(keys = 'id')
train_pheno = pheno[pheno.index.isin(train.index)]
test_pheno = pheno.drop(train.index)
train.reset_index()
test.reset_index()
train_pheno.reset_index()
test_pheno.reset_index()

# NOTE! However, we start without proper test set and will entrust internal validation.
# Why? Well, we do not have so many individuals to spare for the test set.
train = geno

# Print some info about the resulting split
print("Original data:", orig_geno.shape)
print("\t - training set:", train.shape)
print("\t - test set:", test.shape)

# PCA

Before we even start training our autoencoder model, it'd be good to have some benchmark to compare to. Below, we will do a very simple PCA on raw genotypes, not even making use of the kinship matrix.

In [ ]:
embedding = PCA(n_components=2)
pca_embedding = embedding.fit_transform(geno)
x = pca_embedding[:,0]
y = pca_embedding[:,1]
pop = pheno.iloc[:,5]
data = {'x':x, 'y':y, 'pop':pop}
plt.figure(figsize = (9,9))
sns.scatterplot(x='x', y='y', data=data, hue='pop', style='pop', s=100, legend=True)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, markerscale=2)
plt.show()

As you can see, simple PCA separates our individuals into a couple of clear clusters: Africa in the East part of the plot, China and Japa in the NW part and ethnicities of mostly European descent end up in the SW part. While it is easy for PCA to separate Maasai from Yoruba, people of Chinese descent cannot be easily told apart from Japanese using this approach.

In [ ]:
# For Keras framework, we need to convert our data types a bit
train_tensor = train.to_numpy()
test_tensor = test.to_numpy()
original_dim = train_tensor.shape[1]
latent_dim = 2 # dimentionality of our latent space
print(type(train_tensor))

# Autoencoder Model

Below, we will build our autoencoder model using functional interface provided by the Keras framework. Keras, in turn, will talk to Tensorflow framework that enables us to build, train and use neural networks.

In [ ]:
input = keras.Input(shape = (original_dim,))
output = input

h = layers.Dense(units = 500, activation = 'relu')(input)
h = layers.BatchNormalization()(h)
h = layers.Dropout(rate = 0.05)(h)
h = layers.Dense(units = 250, activation = 'relu')(h)
h = layers.Dense(units = 25, activation = 'relu')(h)
latent = layers.Dense(units = latent_dim, name = 'latent')(h)
encoder = keras.Model(input, latent, name='encoder')

latent_inputs = keras.Input(shape=(latent_dim,))
h = layers.Dense(units = 25, activation = 'relu')(latent_inputs)
h = layers.Dense(units = 250, activation = 'relu')(h)
h = layers.Dense(units = 500, activation = 'relu')(h)
dec_output = layers.Dense(original_dim, activation='sigmoid')(h)
decoder = keras.Model(latent_inputs, dec_output, name='decoder')

output = decoder(encoder(input))
ae_model = keras.Model(input, output, name='AE')

Keras.utils provides a very convenient way of visualizing models:

In [ ]:
plot_model(ae_model, show_shapes=True, expand_nested = True, dpi=58)

Once we have checked that the model looks as we wanted it to look, we can compile it. We will change two hyperparameters of the model from their defaults:

*   we set custom loss function. Here, we want our loss function to measure how far the decoded points are from where they are in input. Ideal autoencoder should perfectly reproduce the input on the output,
*   we also use ADAM as our optimizer. Without going into details, this should be your off-the-shelf optimizer in most cases.



In [ ]:
hp_loss_fn = keras.losses.MeanSquaredError()
hp_optimizer = 'adam'
ae_model.compile(
  loss = hp_loss_fn,
  optimizer = hp_optimizer
)
ae_model.summary(expand_nested = True)

Finally, we can train our model:

In [ ]:
hp_epochs = 30
hp_batch_size = 64
hp_val_split = 0.2

autoencoder = ae_model.fit(
                      x = train_tensor,
                      y = train_tensor,
                      epochs = hp_epochs,
                      batch_size = hp_batch_size,
                      validation_split = hp_val_split,
                      verbose = 2
                      )

Once the model is trained, we can have a look at its performance:

In [ ]:
loss = autoencoder.history['loss']
val_loss = autoencoder.history['val_loss']
epochs = range(hp_epochs)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'ro', label='Validation loss')
plt.title('Training and validation loss (MSE)')
plt.legend()
plt.show()

All right, so now our model has been trained and we need to visualize our data in low dimension by using neurons in from the bottleneck layer. The easiest way is to extract the whole trained encoder and run our input through it once again to get embeddings.


In [ ]:
geno_tensor = geno.to_numpy()
geno_dim = geno_tensor.shape[1]
input = keras.Input(shape = (geno_dim,))

trained_encoder = keras.Model(ae_model.input, ae_model.layers[1].get_layer("latent").output)
trained_encoder.summary()


In [ ]:
embedded_points = trained_encoder.predict(geno_tensor)
print(embedded_points)

x = embedded_points[:,0]
y = embedded_points[:,1]
pop = pheno.iloc[:,5]
data = {'x':x, 'y':y, 'pop':pop}
plt.figure(figsize = (10,10))
sns.scatterplot(x='x', y='y', data=data, hue='pop', style='pop', s=100)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, markerscale=2)
plt.show()

## Question 2
Try to see what happens if you add an extra layer with, say 1500 units to the model. What are pros and what are potential cons of such approach?

# VAE

Now it is, perhaps, time to go generative and build a variational variant of our autoencoder.

We will need a special layer here, the so-called sampling layer.

In [ ]:
def sampling(args):
    z_mean, z_log_sigma = args
    #epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0, stddev=1)
    epsilon = tf.random.normal(shape=(tf.shape(z_mean)[0], latent_dim), mean=0, stddev=1)
    return z_mean + tf.exp(z_log_sigma) * epsilon

# Encoder
input = keras.Input(shape = (original_dim,))

h = layers.Dense(units = 1500, activation = 'relu')(input)
h = layers.BatchNormalization()(h)
h = layers.Dense(units = 500, activation = 'relu')(h)
h = layers.Dense(units = 250, activation = 'relu')(h)
h = layers.Dense(units = 25, activation = 'relu')(h)
# Bottleneck
z_mean = layers.Dense(latent_dim, name = 'z_mean')(h)
z_log_sigma = layers.Dense(latent_dim, name = 'z_sigma')(h)
# Lambda layer with specified output shape
z_sampling = layers.Lambda(sampling, name='z_sampling', output_shape=(latent_dim,))([z_mean, z_log_sigma])

encoder = keras.Model(input, [z_mean, z_log_sigma, z_sampling], name='encoder')

# Decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
h = layers.Dense(units = 25, activation = 'relu')(latent_inputs)
h = layers.Dense(units = 250, activation = 'relu')(h)
h = layers.Dense(units = 500, activation = 'relu')(h)
h = layers.Dense(units = 1500, activation = 'relu')(h)
dec_output = layers.Dense(original_dim, activation='sigmoid')(h)
decoder = keras.Model(latent_inputs, dec_output, name='decoder')

# instantiate VAE model
output = decoder(encoder(input)[2])
vae_model = keras.Model(input, output, name='VAE')

We also need a custom loss function that has two constraints:
* reconstructed space have to be as close to input as possible (like in vanilla autoencoder),
* points in our latent points have to be distributed as close to a given distribution (here random normal) as possible (this is what KL divergence measures)

In [ ]:
# Define VAE loss in a custom layer
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_sigma, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        
        # Reconstruction loss
        reconstruction_loss = keras.losses.MeanSquaredError()(inputs, reconstructed) * original_dim
        
        # KL divergence loss
        kl_loss = 1 + z_log_sigma - tf.square(z_mean) - tf.exp(z_log_sigma)
        kl_loss = -0.5 * tf.reduce_sum(kl_loss, axis=-1)
        
        # Add losses to the model
        self.add_loss(tf.reduce_mean(reconstruction_loss + kl_loss))
        return reconstructed

vae_model = VAE(encoder, decoder)
vae_model.compile(optimizer='adam')
vae_model.summary()

In [ ]:
plot_model(encoder, show_shapes=True, show_layer_names = True, dpi = 60)

In [ ]:
plot_model(decoder, show_shapes=True, expand_nested = True, dpi = 60)

In [ ]:
hp_epochs = 30
hp_batch_size = 64
hp_val_split = 0.2

vae_model.fit(train_tensor, train_tensor,
        epochs = hp_epochs,
        batch_size = hp_batch_size,
        shuffle = True,
        validation_split = hp_val_split)

In [ ]:
loss = vae_model.history.history['loss']
val_loss = vae_model.history.history['val_loss']
epochs = range(hp_epochs)
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'ro', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
embedded_points = encoder.predict(geno_tensor)
print(embedded_points)

x = embedded_points[0][:,0]
y = embedded_points[0][:,1]
pop = pheno.iloc[:,5]
data = {'x':x, 'y':y, 'pop':pop}
plt.figure(figsize = (10,10))
sns.scatterplot(x='x', y='y', data=data, hue='pop', style='pop', s=100)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, markerscale=2)
plt.show()

Now, let us create some 10 imaginary individuals around a given point in the latent space, here [0.4, -0.5].

In [ ]:
N_ind = 10
center = [0, 0.0]
cx = tf.random.normal(shape = [N_ind], mean = 0.4, stddev = 0.1)
cy = tf.random.normal(shape = [N_ind], mean = -0.5, stddev = 0.1)
z_sample = np.column_stack((cx, cy))
z_sample

In [ ]:
# Let's thread those artificial individuals through the decoder part of the model
decoded = decoder.predict(z_sample,)
decoded

In [ ]:
# And see what are their genotypes
new_geno = np.zeros(shape = decoded.shape)
new_geno[decoded <= 0.33] = 0
new_geno[np.logical_and(decoded > 0.33, decoded < 0.66)] = 1
new_geno[decoded > 0.66] = 2
print(new_geno)

In [ ]:
# And now, we can see where our individuals ended up in the embedded space. 
x_encoded = encoder.predict(np.row_stack((geno_tensor, decoded)), batch_size=32)
x = x_encoded[0][:, 0]
y = x_encoded[0][:, 1]
pop_list = [pheno['population'], pd.Series(np.repeat('TST', new_geno.shape[0]))]
pop = pd.concat(pop_list)
data_tmp = {'x':x, 'y':y, 'pop':pop}
plt.figure(figsize = (7, 7))
sns.scatterplot(x='x', y='y', data=data_tmp, hue='pop', style='pop', s=100)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0, markerscale=2)
plt.show()

## Question 3
Remove the layer with 1500 units from the VAE model and see how does it affect the result. What do you think is happening here?